In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import joblib

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, Normalizer, RobustScaler, LabelEncoder
from sklearn.metrics import *
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV


import keras

#from keras.utils.np_utils import to_categorical
import os
import scipy as sp

import urllib.request
import shutil
import tarfile

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

import time
import logging
from typing import List, Tuple, Generator, Iterator
import random

np.random.seed(0)
%matplotlib inline
pd.set_option('display.max_columns', None)


from pyod.models.ocsvm import OCSVM
from pyod.models.iforest import IForest

In [24]:
for dirname, _, filenames in os.walk('/Users/nednik/Downloads/Dataset/Entire-Dataset/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_32.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_26.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_27.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_33.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_19.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_25.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_31.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_30.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_24.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_18.csv
/Users/nednik/Downloads/Dataset/Entire-Dataset/.DS_Store
/Users/nednik/Downloads/Dataset/Entire-Dataset/UNSW_2018_IoT_Botnet_Dataset_20.csv
/Users/nednik/Downloads/Datase

In [25]:
import os
import random
import pandas as pd

# Path to the directory containing CSV files
path = '/Users/nednik/Downloads/Dataset/Entire-Dataset/'

# Create a list to store all dataframes
dataframes = []

# Number of samples to read from each file
samples_per_file = 300

# Iterate through all CSV files in the directory
for file_name in os.listdir(path):
    # Process only files with '.csv' extension and skip 'column names'
    if file_name.endswith('.csv') and file_name != 'UNSW_2018_IoT_Botnet_Dataset_Feature_Names.csv':
        file_path = os.path.join(path, file_name)
        
        # Read the entire CSV file into a dataframe
        df = pd.read_csv(file_path, low_memory=False)

# The following code block is commented out. It demonstrates sampling from the dataframe:

        # Get all samples with the category "Normal"
        normal_samples = df[df['category'] == 'Normal']
        
        # Randomly select a specified number of samples with a category other than "Normal"
        non_normal_samples = df[df['category'] != 'Normal']
        random_samples = non_normal_samples.sample(n=samples_per_file, random_state=42)
        
        # Combine the samples into a single dataframe
        merged_df = pd.concat([normal_samples, random_samples])
        
        # Add the merged dataframe to the list
        dataframes.append(merged_df)

# Concatenate all dataframes into one unified dataframe
botiot = pd.concat(dataframes, ignore_index=True)


In [26]:
botiot.loc[(botiot['category'] != 'Normal'), 'category'] = 'Attack'
botiot.loc[(botiot['category'] == 'Normal'), 'category'] = '0_Normal'

In [27]:
botiot['category'].value_counts()

category
Attack      22200
0_Normal     9543
Name: count, dtype: int64

In [28]:
botiot.head()

,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,state,ltime,seq,dur,mean,stddev,smac,dmac,sum,min,max,soui,doui,sco,dco,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
0,31005122,1.528087e+09,e,tcp,192.168.100.3,41307,52.64.239.193,8883,22,8961,CON,1.528087e+09,100945,22.359011,3.541039,1.888227,NaN,NaN,17.705196,0.372132,4.999864,NaN,NaN,NaN,NaN,11,11,8235,726,0.939219,0.499633,0.447247,0,0_Normal,Normal
1,31167372,1.528087e+09,e,udp,192.168.100.46,3456,192.168.100.5,80,4914,4779854,CON,1.528087e+09,1032,30.763821,4.384348,1.490396,NaN,NaN,30.690439,0.733647,4.997457,NaN,NaN,NaN,NaN,2457,2457,2334520,2445334,159.700577,79.834038,79.834038,0,0_Normal,Normal
2,31167373,1.528087e+09,e *,tcp,192.168.100.55,8080,192.168.100.3,80,4914,4787822,CON,1.528087e+09,1033,30.763823,4.383881,1.491516,NaN,NaN,30.687164,0.730435,4.997447,NaN,NaN,NaN,NaN,2456,2458,2469169,2318653,159.700577,79.801529,79.866539,0,0_Normal,Normal
3,31167374,1.528087e+09,e,udp,192.168.100.46,80,192.168.100.5,80,2456,2357989,INT,1.528087e+09,1034,30.751413,4.381677,1.495236,NaN,NaN,30.671740,0.719116,4.996557,NaN,NaN,NaN,NaN,2456,0,2357989,0,79.833733,79.833733,0.000000,0,0_Normal,Normal
4,31167375,1.528087e+09,e,tcp,192.168.100.46,80,192.168.100.5,80,2457,2302694,CON,1.528087e+09,1035,30.751417,4.381808,1.495291,NaN,NaN,30.672657,0.719116,4.997444,NaN,NaN,NaN,NaN,2457,0,2302694,0,79.866241,79.866241,0.000000,0,0_Normal,Normal


In [29]:
botiot=botiot.drop(columns = ['pkSeqID', 'stime', 'ltime', 'seq', 'smac',  'dmac', 'soui', 'doui', 'sco', 'dco', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'srate', 'drate', 'attack', 'subcategory'])

In [30]:
botiot.select_dtypes(include=['object']).columns

Index(['flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'state',
       'category'],
      dtype='object')

In [31]:
botiot.isna().sum()
botiot = botiot.dropna(subset=['sport'])
botiot.drop_duplicates(inplace = True)

In [32]:
botiot = botiot.drop(columns = ['saddr', 'daddr',  'state', 'sport', 'dport'])

In [33]:
# def convert_to_int(value):
#     try:
#         if 'M' in str(value):
#             result = int(float(value[:-2])*10**6)
#         else:
#             result = int(value)
#     except (ValueError, TypeError):
#         print(value)
#         result = 0
#     return result

# botiot['sport'] = botiot['sport'].apply(convert_to_int)
# botiot['dport'] = botiot['dport'].apply(convert_to_int)

In [34]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
botiot['flgs'] = label_encoder.fit_transform(botiot['flgs'])
botiot['proto'] = label_encoder.fit_transform(botiot['proto'])

In [35]:
botiot

,flgs,proto,pkts,bytes,dur,mean,stddev,sum,min,max,rate,category
0,0,2,22,8961,22.359011,3.541039,1.888227,17.705196,0.372132,4.999864,0.939219,0_Normal
1,0,3,4914,4779854,30.763821,4.384348,1.490396,30.690439,0.733647,4.997457,159.700577,0_Normal
2,3,2,4914,4787822,30.763823,4.383881,1.491516,30.687164,0.730435,4.997447,159.700577,0_Normal
3,0,3,2456,2357989,30.751413,4.381677,1.495236,30.671740,0.719116,4.996557,79.833733,0_Normal
4,0,2,2457,2302694,30.751417,4.381808,1.495291,30.672657,0.719116,4.997444,79.866241,0_Normal
...,...,...,...,...,...,...,...,...,...,...,...,...
31738,0,3,5,300,24.996321,1.154677,1.999959,4.618706,0.000000,4.618706,0.160024,Attack
31739,0,3,4,240,11.794052,4.110106,0.743380,8.220211,3.366726,4.853485,0.254366,Attack
31740,0,3,17,1020,31.547817,3.305811,1.747386,16.529053,0.000000,4.647075,0.507167,Attack
31741,0,3,5,300,17.014788,2.754292,2.052948,8.262876,0.000000,4.926610,0.235090,Attack


In [36]:
def prepare_data(data):
    known = "0_Normal"
    
    data_train, data_test = train_test_split(data, test_size=0.3, random_state=__SEED)
    

    # Loại bỏ các samples không phải normal trong tập train
    data_train = data_train[data_train.iloc[:, -1] == known]

    X_train = data_train.iloc[:, :-1]
    X_test = data_test.iloc[:, :-1]
    y_labels = data_test.iloc[:, -1]


    y_test = np.array([1 if cl not in known else 0 for cl in y_labels])

    return X_train, X_test, y_test

In [37]:
def Get_Scaler(name):
  # (StandardScaler, MinMaxScaler, RobustScaler, Normalizer)
  if name == "StandardScaler":
    return StandardScaler()
  if name == "MinMaxScaler":
    return MinMaxScaler()
  if name == "RobustScaler":
    return RobustScaler()
  if name == "Normalizer":
    return Normalizer()
  if name == "QuantileTransformer":
      return QuantileTransformer(output_distribution = "Normal", random_state=__SEED)
  return None

def preprocess_data(data, save_scaler_path='./saved_model/scaler.pkl'):
    X_train, X_test, y_test = prepare_data(data)

    ## ========================== Scaler data ================================================
    scaler = Get_Scaler(name=__SCALER)
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # Save the scaler
    joblib.dump(scaler, save_scaler_path)
    print(f"Scaler saved to {save_scaler_path}")

    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test counts: {np.unique(y_test, return_counts=True)}")

    return X_train, X_test, y_test

In [38]:
import os
import pickle

def model(data):
    # Preprocess the data
    X_train, X_test, y_test = preprocess_data(data)

    # Isolation Forest Hyperparameter Optimization
    iforest_params = {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'contamination': [0.1, 0.2, 0.3],
        'random_state': [42]
    }
    iforest = IsolationForest()
    iforest_grid = GridSearchCV(iforest, iforest_params, scoring='accuracy', cv=3)
    iforest_grid.fit(X_train, np.zeros(X_train.shape[0]))  # Fit unsupervised

    # Best Model
    best_iforest = iforest_grid.best_estimator_
    iforest_pred = best_iforest.predict(X_test)
    iforest_pred = [1 if p == -1 else 0 for p in iforest_pred]  # Convert -1 to 1 (anomaly) and 1 to 0

    # Save the best Isolation Forest model
    save_dir = './saved_model'
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it does not exist
    save_path = os.path.join(save_dir, 'iforest_model.pkl')

    with open(save_path, 'wb') as f:
        pickle.dump(best_iforest, f)
    print(f"Isolation Forest model saved to {save_path}")

    # Evaluate the model
    iforest_acc, iforest_precision, iforest_recall, iforest_f1, iforest_cnf_matrix = Evaluate_Model(y_test, iforest_pred)

    # One-Class SVM for Comparison
    ocsvm_clf = OCSVM()
    ocsvm_clf.fit(X_train)
    ocsvm_pred = ocsvm_clf.predict(X_test)
    ocsvm_acc, ocsvm_precision, ocsvm_recall, ocsvm_f1, ocsvm_cnf_matrix = Evaluate_Model(y_test, ocsvm_pred)

    # Collecting Results
    models = ['Optimized IForest', 'OCSVM']
    parameters = [iforest_grid.best_params_, ocsvm_clf.get_params()]
    accuracies = [iforest_acc, ocsvm_acc]
    precisions = [iforest_precision, ocsvm_precision]
    recalls = [iforest_recall, ocsvm_recall]
    f1_scores = [iforest_f1, ocsvm_f1]
    tprs = [iforest_recall, ocsvm_recall]
    fprs = [
        iforest_cnf_matrix[0, 1] / (iforest_cnf_matrix[0, 1] + iforest_cnf_matrix[0, 0]),
        ocsvm_cnf_matrix[0, 1] / (ocsvm_cnf_matrix[0, 1] + ocsvm_cnf_matrix[0, 0])
    ]
    matrix = [iforest_cnf_matrix, ocsvm_cnf_matrix]

    # Creating Results DataFrame
    results = {
        'Model': models,
        'Best Params': parameters,
        'Accuracy': accuracies,
        'Precision': precisions,
        'Recall': recalls,
        'F1 Score': f1_scores,
        'TPR': tprs,
        'FPR': fprs,
        'Confusion Matrix': matrix
    }

    df_results = pd.DataFrame(results)

    return df_results


In [39]:
from sklearn.metrics import matthews_corrcoef

def Evaluate_Model(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cnf_matrix = confusion_matrix(y_test, y_pred)

    return acc, precision, recall, f1, cnf_matrix

In [40]:
__SEED = 42
__SCALER = "StandardScaler"

In [41]:
results_botiot = model(botiot)
results_botiot

Scaler saved to ./saved_model/scaler.pkl
X_train shape: (6166, 11)
X_test shape: (9301, 11)
y_test counts: (array([0, 1]), array([2639, 6662]))
Isolation Forest model saved to ./saved_model/iforest_model.pkl


,Model,Best Params,Accuracy,Precision,Recall,F1 Score,TPR,FPR,Confusion Matrix
0,Optimized IForest,"{'contamination': 0.1, 'max_samples': 0.5, 'n_...",0.871519,0.955507,0.860702,0.905631,0.860702,0.101175,"[[2372, 267], [928, 5734]]"
1,OCSVM,"{'cache_size': 200, 'coef0': 0.0, 'contaminati...",0.657564,0.933001,0.562294,0.701695,0.562294,0.101933,"[[2370, 269], [2916, 3746]]"


In [42]:
import joblib
import os

# Function to save the trained model and scaler
def save_model_and_scaler(model, save_dir='./saved_model'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Paths to save the model and scaler
    model_path = os.path.join(save_dir, 'trained_model.pkl')
    
    # Save the model and scaler
    joblib.dump(model, model_path)
    
    print(f"Model saved to: {model_path}")
    return model_path


In [43]:
# Example usage: Call this function after training the model
save_model_and_scaler(model)

Model saved to: ./saved_model/trained_model.pkl


'./saved_model/trained_model.pkl'

In [44]:
print("Training Data Stats:")
print(botiot.describe())

Training Data Stats:
               flgs         proto           pkts         bytes           dur  \
count  31003.000000  31003.000000   31003.000000  3.100300e+04  31003.000000   
mean       1.986582      2.615844     660.609489  5.632098e+05     43.043330   
std        3.099777      0.491220    6853.502966  6.449798e+06    203.930573   
min        0.000000      0.000000       1.000000  6.000000e+01      0.000000   
25%        0.000000      2.000000       2.000000  2.400000e+02      5.045178   
50%        0.000000      3.000000       5.000000  4.800000e+02     17.431620   
75%        7.000000      3.000000       8.000000  7.360000e+02     31.736982   
max        8.000000      3.000000  322677.000000  3.144596e+08   3536.089844   

               mean        stddev           sum           min           max  \
count  31003.000000  31003.000000  31003.000000  31003.000000  31003.000000   
mean       1.665777      0.648293     23.737298      0.826194      2.367822   
std        1.737147  